## Table of Contents
<ul>
<li><a href="#introduction">Introduction</a></li>
<li><a href="#assess">Data Assessment</a></li>
<li><a href="#cleaning">Data Cleaning/Preprocessing</a></li>
<li><a href="#model">Models</a></li>
<li><a href="#Results">Results/Findings</a></li>
<li><a href="#sumbit">Submission</a></li>
</ul>

## Preliminary Wrangling

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder

sns.set()
%matplotlib inline

In [ ]:
# Loading train and test dataset
df_train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
df_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
df = pd.concat([df_train, df_test]).reset_index(drop=True)
df

<a id='assess'></a>
### Data Assessment

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.describe().T[df.describe().T['min']==0].index

### Visualizing Numerical Columns

In [ ]:
sns.set(font_scale = 1.5)
df.drop('Id', axis=1).hist(figsize = (30, 30));

**Insight**
- The columns listed below has a min value equal to 0. This Zero also mean unavailable:
>MasVnrArea, BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, 2ndFlrSF, LowQualFinSF, BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, BedroomAbvGr, KitchenAbvGr, Fireplaces, GarageCars, GarageArea, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea, MiscVal.

In [ ]:
# Sorting columns with null values
columns_null = [cols for cols in df.columns if df[cols].isnull().sum() > 0]
df[columns_null].isnull().sum().sort_values(ascending=False)

### **`Data Assessment Results`**
- **Fill NaN with 0:** LotFrontage, MasVnrArea, GarageYrBlt, BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, BsmtFullBath, BsmtHalfBath, GarageCars, GarageArea
- **MasVnrType:** Convert None to NaN
- **Label Encode:** Street, Utilities, CentralAir
- **One-Hot Encode:** MSZoning, Alley, LotShape, LandContour, LotConfig, LandSlope, Neighborhood, Condition1, Condition2, BldgType, HouseStyle, RoofStyle, RoofMatl, Exterior1st, Exterior2nd, MasVnrType, ExterQual, ExterCond, Foundation, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, Heating, HeatingQC, Electrical, KitchenQual, Functional, FireplaceQu, GarageType, GarageFinish, GarageQual, GarageCond, PavedDrive, PoolQC, Fence, MiscFeature, SaleType, SaleCondition

<a id='cleaning'></a>
### Data Cleaning/Preprocessing
> We would first make a copy the dataset, then perform variou cleaning/ preprocessind operation on the dataset

##### **Fill NaN with 0:** LotFrontage, MasVnrArea, GarageYrBlt, BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, BsmtFullBath, BsmtHalfBath, GarageCars, GarageArea

In [ ]:
zero_list = ['LotFrontage', 'MasVnrArea', 'GarageYrBlt', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageCars', 'GarageArea']

for col in zero_list:
    df[col].fillna(0, inplace = True)
    # Print to confirm
    print(col, df[col].isnull().sum())

##### MasVnrType : Convert None to NaN then one hot

In [ ]:
# Converting 'None' to NaN
# df['MasVnrType'] = df['MasVnrType'].replace('None', np.nan)

# Checking Result
df['MasVnrType'].unique()

#### **Label Encode**
> Street, Utilities, CentralAir

In [ ]:
binary_columns = [col for col in df.columns if df[col].dtype == "object" and len(df[col].unique()) == 2]
print(binary_columns)

In [ ]:
def label_encoding(df,col):
    label_enc = LabelEncoder()
    df[col] = label_enc.fit_transform(df[col])
    return df

# Label Encoding binary columns
for col in binary_columns:
    df = label_encoding(df,col)

    
# Checking Result
df[binary_columns].sample(5)

#### **One-Hot Encode**
> MSZoning, Alley, LotShape, LandContour, LotConfig, LandSlope, Neighborhood, Condition1, Condition2, BldgType, HouseStyle, RoofStyle, RoofMatl, Exterior1st, Exterior2nd, MasVnrType, ExterQual, ExterCond, Foundation, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, Heating, HeatingQC, Electrical, KitchenQual, Functional, FireplaceQu, GarageType, GarageFinish, GarageQual, GarageCond, PavedDrive, PoolQC, Fence, MiscFeature, SaleType, SaleCondition

In [ ]:
one_hot_cols = [cols for cols in df.columns if df[cols].dtype == 'object' and len(df[cols].unique()) > 2]
df = pd.get_dummies(df, columns= one_hot_cols)
# Checking Results
df.head()

<a id='model'></a>
## **MODELS**

#### Importing Libraries

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, ElasticNet, LassoCV, RidgeCV, ElasticNetCV

#### Define a function to return root-mean squared error(RMSE)

In [ ]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

#### Splitting Data into X(features) and Y(target)

In [ ]:
X = df.drop(columns=['Id', 'SalePrice'], axis=1)
Y = df['SalePrice']

#### Split Data into train and test data

In [ ]:
X_train_val = X.loc[:1459, :]
Y_train_val = Y.loc[:1459]
X_test = X.loc[1460:, :]
Y_test = Y.loc[1460:]

X_train, X_val, Y_train, Y_val = train_test_split(X_train_val, Y_train_val, random_state=19, test_size=0.1)

### Linear Regression

In [ ]:
lin_model = LinearRegression()
lin_model.fit(X_train,Y_train)

lin_pred= lin_model.predict(X_val)
lin_rmse = rmse(Y_val, lin_pred)
print('RMSE:', lin_rmse)

### Lasso Regression

In [ ]:
alpha1 = [0.005, 0.05, 0.1, 0.3, 1, 5, 20, 50, 80, 100, 120, 140]

lassorCV = LassoCV(alphas=alpha1, max_iter= 5e4,cv=5)
lassorCV.fit(X_train, Y_train)

lassorCV_rmse = rmse(Y_val, lassorCV.predict(X_val))
print('alpha:',lassorCV.alpha_)
print('RMSE:',lassorCV_rmse)

### Ridge Regression

In [ ]:
alpha2 = [0.005, 0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 30, 50, 80, 100]

ridgeCV = RidgeCV(alphas=alpha2, cv=5)
ridgeCV.fit(X_train, Y_train)

ridgeCV_rmse = rmse(Y_val, ridgeCV.predict(X_val))
print('alpha:',ridgeCV.alpha_)
print('RMSE:',ridgeCV_rmse)

### ElasticNet Regression

In [ ]:
l1_ratios = np.linspace(0.1, 0.9, 9)

elasticNetCV = ElasticNetCV(alphas=alpha1, l1_ratio=l1_ratios,
                            max_iter= 1e4,cv=5)
elasticNetCV.fit(X_train, Y_train)

elasticNetCV_rmse = rmse(Y_val, elasticNetCV.predict(X_val))
print('alpha:',elasticNetCV.alpha_)
print('l1 ratio:', elasticNetCV.l1_ratio_)
print('RMSE:',elasticNetCV_rmse)

### LightGBM Regression

In [ ]:
from lightgbm import LGBMRegressor

lgbmRegressor = LGBMRegressor()
lgbmRegressor.fit(X_train, Y_train)
lgbm_rmse = rmse(Y_val, lgbmRegressor.predict(X_val))
print('RMSE:', lgbm_rmse)

### Gradient Boosting Regression

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbRegressor = GradientBoostingRegressor()
gbRegressor.fit(X_train, Y_train)
gb_rmse = rmse(Y_val, gbRegressor.predict(X_val))
print('RMSE:', gb_rmse)

### CatBoost Regression

In [ ]:
from catboost import CatBoostRegressor

cbRegressor = CatBoostRegressor()
cbRegressor.fit(X_train, Y_train, silent=True)
cb_rmse = rmse(Y_val, cbRegressor.predict(X_val))
print('RMSE:', cb_rmse)

### Bayesian Ridge

In [ ]:
from sklearn.linear_model import BayesianRidge

brRegressor = BayesianRidge()
brRegressor.fit(X_train, Y_train)
br_rmse = rmse(Y_val, brRegressor.predict(X_val))
print('RMSE:', br_rmse)

## Random Forest Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfRegressor = RandomForestRegressor(random_state=19)
rfRegressor.fit(X_train, Y_train)
rf_rmse = rmse(Y_val, rfRegressor.predict(X_val))
print('RMSE:', rf_rmse)

### Extra Trees Regressor

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

extRegressor = ExtraTreesRegressor(random_state=19)
extRegressor.fit(X_train, Y_train)
ext_rmse = rmse(Y_val, extRegressor.predict(X_val))
print('RMSE:', ext_rmse)

<a id='Results'></a>
## Results/Finding

### Comparing the RMSE calculation from all models

In [ ]:
rmse_val =  [lin_rmse, lassorCV_rmse, ridgeCV_rmse, elasticNetCV_rmse, lgbm_rmse, 
             gb_rmse, cb_rmse, br_rmse, rf_rmse, ext_rmse]
labels = ['Linear', 'Lasso', 'Ridge', 'ElasticNet', 'LightGBM', 'Gradient Boosting', 
         'CatBoost', 'Bayesian Ridge', 'Random Forest', 'Extra Trees']

rmse_df = pd.Series(rmse_val, index=labels).to_frame()
rmse_df.rename(columns={0 : 'RMSE'}, inplace=True)
rmse_df.sort_values(by='RMSE')

**RESULT:** The best model for is the model with the lowest RMSE score which is the **CatBoost regression model**.

<a id='sumbit'></a>
## Submission

In [ ]:
output = pd.DataFrame(columns=["Id","SalePrice"])
output["Id"] = df_test.Id
output["SalePrice"] = cbRegressor.predict(X_test)
output

In [ ]:
output.to_csv('submission.csv', index=False)
print('Submission succesful!')